<a href="https://colab.research.google.com/github/yukinaga/hopfield_boltzmann/blob/main/section_3/03_rbm_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 制限ボルツマンマシンをPythonのクラスで実装する
可視層 (Visible Layer) と 隠れ層 (Hidden Layer) の2層構造を持つ制限ボルツマンマシン (RBM) を、Pythonのクラスで実装します。  
こRBM のクラス `RestrictedBoltzmannMachine` がどのように動作し、各メソッドが何をしているのかを解説します。

## BRestrictedBoltzmannMachineクラス
- **可視層 (Visible Layer)** と **隠れ層 (Hidden Layer)** の2層構造を持つ制限ボルツマンマシン (RBM) を実装しています。  
- **Contrastive Divergence (CD)** アルゴリズムを用いて、データから重みとバイアスを学習します。  
- 温度パラメータを導入して、学習時の探索の幅を調整します。  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score

class RestrictedBoltzmannMachine:
    def __init__(self, n_visible, n_hidden, learning_rate=0.1, temperature=1.0, epochs=1000):
        """RBMの初期化"""
        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.learning_rate = learning_rate
        self.temperature = temperature  # 温度パラメータ
        self.epochs = epochs

        # 重み行列の初期化（標準正規分布で初期化）
        self.weights = np.random.randn(n_hidden, n_visible) * 0.1
        # バイアスの初期化
        self.visible_bias = np.zeros(n_visible)
        self.hidden_bias = np.zeros(n_hidden)

    def sigmoid(self, x):
        """温度パラメータを考慮したシグモイド関数"""
        return 1 / (1 + np.exp(-x / self.temperature))

    def sample_hidden(self, visible):
        """可視層から隠れ層へのサンプリング"""
        activation = np.dot(visible, self.weights.T) + self.hidden_bias
        prob_hidden = self.sigmoid(activation)
        hidden_state = (np.random.rand(self.n_hidden) < prob_hidden).astype(np.int_)
        return prob_hidden, hidden_state

    def sample_visible(self, hidden):
        """隠れ層から可視層へのサンプリング"""
        activation = np.dot(hidden, self.weights) + self.visible_bias
        prob_visible = self.sigmoid(activation)
        visible_state = (np.random.rand(self.n_visible) < prob_visible).astype(np.int_)
        return prob_visible, visible_state

    def contrastive_divergence(self, input_data):
        """Contrastive Divergence法でRBMを学習"""
        prob_hidden, hidden_state = self.sample_hidden(input_data)
        prob_visible, visible_state = self.sample_visible(hidden_state)
        prob_hidden_reconstructed, _ = self.sample_hidden(visible_state)

        # 勾配計算と重み更新
        positive_grad = np.outer(prob_hidden, input_data)
        negative_grad = np.outer(prob_hidden_reconstructed, visible_state)

        self.weights += self.learning_rate * (positive_grad - negative_grad)
        self.visible_bias += self.learning_rate * (input_data - visible_state)
        self.hidden_bias += self.learning_rate * (prob_hidden - prob_hidden_reconstructed)

    def train(self, data):
        """RBMのトレーニング"""
        for epoch in range(self.epochs):
            for sample in data:
                self.contrastive_divergence(sample)

            # 温度を徐々に下げる（アニーリング）
            self.temperature *= 0.99

            if (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch + 1}/{self.epochs}, Temperature: {self.temperature:.4f}")

    def run_hidden(self, data):
        """可視層から隠れ層の表現を取得"""
        _, hidden_state = self.sample_hidden(data)
        return hidden_state

    def run_visible(self, hidden):
        """隠れ層から可視層を再構成"""
        _, visible_state = self.sample_visible(hidden)
        return visible_state

以下、各メソッドを解説します。

### **1. `__init__` メソッド**
```python
def __init__(self, n_visible, n_hidden, learning_rate=0.1, temperature=1.0, epochs=1000):
    """RBMの初期化"""
    self.n_visible = n_visible
    self.n_hidden = n_hidden
    self.learning_rate = learning_rate
    self.temperature = temperature  # 温度パラメータ
    self.epochs = epochs

    # 重み行列の初期化（正規分布に従うランダムな小さい値で初期化）
    self.weights = np.random.randn(n_hidden, n_visible) * 0.1
    # 可視層と隠れ層のバイアス初期化（0に初期化）
    self.visible_bias = np.zeros(n_visible)
    self.hidden_bias = np.zeros(n_hidden)
```

#### **解説**
- **`n_visible`**: 可視層のノード数（例えば、MNIST の場合は 28×28 = 784）。
- **`n_hidden`**: 隠れ層のノード数（抽出する特徴の数）。
- **`learning_rate`**: 重みやバイアスの更新の際の学習率。
- **`temperature`**: 温度パラメータ。値が大きいと確率が平坦になり、探索的な学習になります。値が小さいと確率が鋭くなり、確定的な学習になります。
- **`epochs`**: 学習のエポック数。
- **`weights`**: 可視層と隠れ層の間の重み行列（`n_hidden × n_visible`）。
- **`visible_bias`**: 可視層のバイアス。
- **`hidden_bias`**: 隠れ層のバイアス。

---

### **2. `sigmoid` メソッド**
```python
def sigmoid(self, x):
    """温度パラメータを考慮したシグモイド関数"""
    return 1 / (1 + np.exp(-x / self.temperature))
```

#### **解説**
- **シグモイド関数**は、入力値 `x` を 0 から 1 の範囲の確率に変換します。
- 温度パラメータ `self.temperature` を使用して、関数の傾きを調整します。

---

### **3. `sample_hidden` メソッド**
```python
def sample_hidden(self, visible):
    """可視層から隠れ層へのサンプリング"""
    activation = np.dot(visible, self.weights.T) + self.hidden_bias
    prob_hidden = self.sigmoid(activation)
    hidden_state = (np.random.rand(self.n_hidden) < prob_hidden).astype(np.int_)
    return prob_hidden, hidden_state
```

#### **解説**
- **目的**: 可視層の入力から隠れ層のノードをサンプリングする。
- **手順**:
  1. **活性化関数の計算**:
     - `activation = np.dot(visible, self.weights.T) + self.hidden_bias`
     - 可視層の入力と重み行列の内積を計算し、隠れ層のバイアスを加えます。
  2. **シグモイド関数を通して確率に変換**。
  3. **確率に基づいて0または1にサンプリング**します（`np.random.rand()` を用いた確率的サンプリング）。

---

### **4. `sample_visible` メソッド**
```python
def sample_visible(self, hidden):
    """隠れ層から可視層へのサンプリング"""
    activation = np.dot(hidden, self.weights) + self.visible_bias
    prob_visible = self.sigmoid(activation)
    visible_state = (np.random.rand(self.n_visible) < prob_visible).astype(np.int_)
    return prob_visible, visible_state
```

#### **解説**
- **目的**: 隠れ層の状態から可視層のノードをサンプリングする。
- **手順**は `sample_hidden` メソッドと同様です。

---

### **5. `contrastive_divergence` メソッド**
```python
def contrastive_divergence(self, input_data):
    """Contrastive Divergence法でRBMを学習"""
    prob_hidden, hidden_state = self.sample_hidden(input_data)
    prob_visible, visible_state = self.sample_visible(hidden_state)
    prob_hidden_reconstructed, _ = self.sample_hidden(visible_state)

    # 勾配計算と重み更新
    positive_grad = np.outer(prob_hidden, input_data)
    negative_grad = np.outer(prob_hidden_reconstructed, visible_state)

    self.weights += self.learning_rate * (positive_grad - negative_grad)
    self.visible_bias += self.learning_rate * (input_data - visible_state)
    self.hidden_bias += self.learning_rate * (prob_hidden - prob_hidden_reconstructed)
```

#### **解説**
- **Contrastive Divergence (CD)** を使用して RBM のパラメータを更新します。
- **手順**:
  1. **正方向パス**: 入力から隠れ層の状態を計算。
  2. **負方向パス**: 隠れ層から可視層を再構成し、再度隠れ層の状態を計算。
  3. **重みの更新**:
     - 正方向パスと負方向パスの勾配の差を用いて重みとバイアスを更新。

---

### **6. `train` メソッド**
```python
def train(self, data):
    """RBMのトレーニング"""
    for epoch in range(self.epochs):
        for sample in data:
            self.contrastive_divergence(sample)
        
        # 温度を徐々に下げる（アニーリング）
        self.temperature *= 0.99
        
        if (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}/{self.epochs}, Temperature: {self.temperature:.4f}")
```

#### **解説**
- **データセット全体に対して Contrastive Divergence を繰り返し適用**し、RBM の重みとバイアスを学習します。
- 温度パラメータを **エポックごとに少しずつ減少**（アニーリング）させることで、モデルの収束を促します。

---

### **7. `run_hidden` および `run_visible` メソッド**
```python
def run_hidden(self, data):
    """可視層から隠れ層の表現を取得"""
    _, hidden_state = self.sample_hidden(data)
    return hidden_state

def run_visible(self, hidden):
    """隠れ層から可視層を再構成"""
    _, visible_state = self.sample_visible(hidden)
    return visible_state
```

#### **解説**
- **`run_hidden`**: 学習した RBM から、可視層の入力に対して隠れ層の特徴ベクトルを取得。
- **`run_visible`**: 隠れ層の特徴ベクトルから、元の可視層の再構成を行う。